In [1]:
import os
from collections import OrderedDict 
import copy
from pprint import pprint

import pandas as pd
from pandas.tseries.offsets import MonthEnd # can be used for monthly date calculations
import numpy as np

import statsmodels.api as sm

# settings for wide printing of outputs
pd.set_option('display.max_columns',20,'display.width',200)


In [2]:
# ******************************************************
# Enter the path of your data file here
# ******************************************************


In [3]:
# ******************************************************
# DO NOT CHANGE THIS CELL - THIS RETRIEVES YOUR DATA
# ******************************************************

RANDOM_SEED = 0
NUM_ASSET = 80

# market data as of 20201231
df_snap = pd.read_csv(os.path.join('equity-snapshot-20201231.csv'),header=0,index_col=0)

# monthly return history
df_ret_mthly = pd.read_csv(os.path.join('mthlyret60-20201231.csv'),header=0)

# benchmark - the "fin484 index" constituents
df_fin485 = pd.read_csv(os.path.join('fin485.csv'),header=0)

# portfolio - equal weighted random portfolio form within benchmark
np.random.seed(RANDOM_SEED)
tickers = np.random.choice(df_fin485['TICKER'], size=NUM_ASSET, replace=False)
df_port = pd.DataFrame(tickers,columns=['TICKER'])
df_port['PORTWT'] = 1/len(df_port)

# fama french data
df_ff3 = pd.read_csv(os.path.join('ff3.csv'),header=0,index_col=0)

print(df_snap.head())
print(df_ret_mthly.head())
print(df_fin485.head())
print(df_port.head())
print(df_ff3.head())


        TICKER      COMPANY_NAME        SECTOR    PRICE       SHSO       MKTCAP  NUM_MOS
MONTH                                                                                   
2020-12   AAPL             APPLE    Technology   132.69  17001.800  2255.968842       60
2020-12   MSFT         MICROSOFT    Technology   222.42   7560.500  1681.606410       60
2020-12   AMZN        AMAZON.COM  Cons_Discret  3256.93    501.751  1634.167884       60
2020-12   TSLA             TESLA  Cons_Discret   705.67    947.901   668.905299       60
2020-12     FB  FACEBOOK CLASS A    Technology   273.16   2403.970   656.668445       60
  TICKER    MONTH      RET
0      A  2020-12   1.3601
1      A  2020-11  14.5068
2      A  2020-10   1.3158
3      A  2020-09   0.5178
4      A  2020-08   4.2458
  TICKER
0      A
1    AAL
2    AAP
3   AAPL
4   ABBV
  TICKER  PORTWT
0    AEP  0.0125
1   NLSN  0.0125
2     EA  0.0125
3     PM  0.0125
4   CINF  0.0125
         Mkt-RF   SMB   HML    RF
MONTH                    

In [4]:
df_ret_mthly_port = df_ret_mthly[df_ret_mthly['TICKER'].isin(df_port['TICKER'])]
df_ret_mthly_port

,TICKER,MONTH,RET
120,AAL,2020-12,11.6065
121,AAL,2020-11,25.2660
122,AAL,2020-10,-8.2181
123,AAL,2020-09,-5.8237
124,AAL,2020-08,17.3561
...,...,...,...
194995,ZTS,2016-05,0.8293
194996,ZTS,2016-04,6.3105
194997,ZTS,2016-03,7.9640
194998,ZTS,2016-02,-4.6225


In [5]:
df_bmk = pd.DataFrame(df_fin485['TICKER'], columns=['TICKER'])
df_bmk = df_bmk.merge(df_snap, how = 'left')
df_bmk['BMKTWT'] = df_bmk['MKTCAP']/df_bmk['MKTCAP'].sum()
df_bmk = df_bmk.drop(['COMPANY_NAME','SECTOR','PRICE','SHSO','MKTCAP','NUM_MOS'],axis=1)
df_bmk

,TICKER,BMKTWT
0,A,0.001114
1,AAL,0.000292
2,AAP,0.000327
3,AAPL,0.069100
4,ABBV,0.005794
...,...,...
480,YUM,0.001003
481,ZBH,0.000978
482,ZBRA,0.000628
483,ZION,0.000218


In [6]:
df_port = df_port.sort_values('TICKER')
df_port

,TICKER,PORTWT
46,AAL,0.0125
41,ADI,0.0125
31,ADP,0.0125
0,AEP,0.0125
9,AKAM,0.0125
...,...,...
38,WYNN,0.0125
36,XOM,0.0125
35,XRAY,0.0125
21,ZBH,0.0125


In [7]:
df_digest = df_bmk.merge(df_port, how = 'left', on = 'TICKER')
df_digest = df_digest.fillna(0)
df_digest['ACTWT'] = df_digest['PORTWT'] - df_digest['BMKTWT'] 
df_digest

,TICKER,BMKTWT,PORTWT,ACTWT
0,A,0.001114,0.0000,-0.001114
1,AAL,0.000292,0.0125,0.012208
2,AAP,0.000327,0.0000,-0.000327
3,AAPL,0.069100,0.0000,-0.069100
4,ABBV,0.005794,0.0000,-0.005794
...,...,...,...,...
480,YUM,0.001003,0.0000,-0.001003
481,ZBH,0.000978,0.0125,0.011522
482,ZBRA,0.000628,0.0000,-0.000628
483,ZION,0.000218,0.0000,-0.000218


In [8]:
#portfolio weight cell 
port_weight_matrix = df_digest['PORTWT'].values
port_weight_matrix = port_weight_matrix.reshape(485,1)

#benchmark weight cell
bmk_weight_matrix = df_digest['BMKTWT'].values
bmk_weight_matrix = bmk_weight_matrix.reshape(485,1)

#active weight cell
act_weight_matrix = df_digest['ACTWT'].values
act_weight_matrix = act_weight_matrix.reshape(485,1)

In [9]:
df_p = pd.pivot_table(df_ret_mthly, index='MONTH', columns='TICKER', values='RET')
df_p.shape

(60, 3253)

In [10]:
df_port_ret=df_p[df_digest['TICKER']]
df_port_ret = df_port_ret.merge(df_ff3,how = 'left', on = 'MONTH',suffixes = ('','_x'))
df_port_ret

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XYL,YUM,ZBH,ZBRA,ZION,ZTS,Mkt-RF,SMB,HML,RF_x
MONTH,,,,,,,,,,,,,,,,,,,,,
2016-01,-9.9498,-7.9339,1.0232,-7.5242,-6.2940,-13.6438,-5.4829,-15.1763,0.9952,-5.1203,...,-1.5069,-0.2707,-3.2460,-13.2807,-16.9231,-9.9652,-5.77,-3.36,2.01,0.01
2016-02,-0.7968,5.4509,-2.3742,-0.1330,-0.5282,-2.9034,-6.2346,2.3514,-5.0028,-4.4654,...,4.4969,0.1382,-2.4682,2.2848,-5.7362,-4.6225,-0.08,0.83,-0.59,0.02
2016-03,6.6934,0.0244,8.0575,12.7211,4.5962,-0.0808,18.4977,7.9762,15.1007,10.1586,...,9.3290,12.9433,10.3934,11.6866,13.5553,7.9640,6.96,0.82,1.11,0.02
2016-04,2.9780,-15.4109,-2.6444,-13.9921,7.8234,-1.6753,2.4575,-6.4468,-1.2128,0.4478,...,2.1516,-2.2535,8.5717,-9.3333,13.6720,6.3105,0.92,0.69,3.28,0.01
2016-05,12.1456,-7.7467,-1.4478,7.1799,3.1639,-11.4927,2.2339,1.8766,5.3578,5.5721,...,7.2739,3.1800,5.4764,-15.1055,2.0500,0.8293,1.78,-0.27,-1.60,0.01
2016-06,-3.0835,-11.2817,5.1048,-4.2660,-1.6208,5.7882,10.0493,-0.8075,-4.7743,-3.6996,...,-0.0224,1.0111,-1.2173,-5.6675,-10.3141,0.2892,-0.05,0.64,-1.40,0.02
2016-07,8.4536,25.3974,5.0919,9.0063,7.9193,7.4004,7.9422,14.5439,-0.4237,2.1610,...,7.0773,8.4175,8.9384,5.8084,10.9431,6.3422,3.95,2.60,-1.23,0.02
2016-08,-2.3488,2.5589,-7.3472,2.3618,-3.2161,2.4796,-0.0254,-6.1006,1.9413,4.5473,...,6.7027,1.4426,-1.1667,31.9751,10.0267,1.2483,0.50,1.13,3.31,0.02
2016-09,0.4789,0.8540,-5.2096,6.5504,-1.6069,-7.1174,9.0215,0.6425,6.2348,6.0893,...,3.1262,0.1102,0.5056,-0.5003,1.4057,1.7808,0.25,2.04,-1.21,0.02


In [11]:
#correlation cell
kho = df_port_ret[['Mkt-RF','SMB','HML']].corr().values
kho

array([[1.        , 0.4453042 , 0.24104724],
       [0.4453042 , 1.        , 0.25000604],
       [0.24104724, 0.25000604, 1.        ]])

In [12]:
TICKERS = df_digest['TICKER']

In [13]:
# beta matrix and residual std matrix cell
beta_matrix = []
resid_matrix = []
for ticker in TICKERS:
    df_port_ret[ticker] = df_port_ret[ticker] - df_port_ret['RF_x']
    port_fitted = sm.OLS(df_port_ret[[ticker]],sm.add_constant(df_port_ret[['Mkt-RF','SMB','HML']])).fit()
    d = [port_fitted.params[1],port_fitted.params[2],port_fitted.params[3]]
    z = port_fitted.resid.std()
    beta_matrix.append(d)
    resid_matrix.append(z)
beta_matrix= np.asarray(beta_matrix)
resid_matrix = np.diag(resid_matrix)
beta_matrix.shape

(485, 3)

In [14]:
df_digest['beta_mkt'] = beta_matrix[:,0].reshape(485,1)
df_digest['beta_smb'] = beta_matrix[:,1].reshape(485,1)
df_digest['beta_hml'] = beta_matrix[:,2].reshape(485,1)
df_digest

,TICKER,BMKTWT,PORTWT,ACTWT,beta_mkt,beta_smb,beta_hml
0,A,0.001114,0.0000,-0.001114,1.090569,-0.278011,-0.273262
1,AAL,0.000292,0.0125,0.012208,1.514827,0.267377,0.829468
2,AAP,0.000327,0.0000,-0.000327,0.983922,0.557808,0.524153
3,AAPL,0.069100,0.0000,-0.069100,1.426600,-0.473334,-0.727169
4,ABBV,0.005794,0.0000,-0.005794,0.670140,0.535816,0.145728
...,...,...,...,...,...,...,...
480,YUM,0.001003,0.0000,-0.001003,1.081674,-0.442271,0.135099
481,ZBH,0.000978,0.0125,0.011522,1.280535,-0.089538,-0.085014
482,ZBRA,0.000628,0.0000,-0.000628,1.409412,0.747161,0.064243
483,ZION,0.000218,0.0000,-0.000218,1.165127,0.986018,1.273312


In [15]:
factors_std = df_port_ret[['Mkt-RF','SMB','HML']].std().values
factors_std_matrix = np.diag(factors_std) * np.sqrt(12)
factors_std_matrix

array([[16.24653641,  0.        ,  0.        ],
       [ 0.        ,  8.95804042,  0.        ],
       [ 0.        ,  0.        , 11.44227463]])

In [16]:
#
#
# Calc Summary Method
# (You will enter code in one section of this method)
#
#

Beta = beta_matrix
Sigma_F = factors_std_matrix
Rho_F = kho
Sigma_epsilon = resid_matrix@resid_matrix


def calc_summary(summary_label, w, Beta, Sigma_F, Rho_F, Sigma_epsilon):
    beta_mkt = (w * df_digest['beta_mkt']).sum().round(3)
    beta_smb = (w * df_digest['beta_smb']).sum().round(3)
    beta_hml = (w * df_digest['beta_hml']).sum().round(3)
    w = w.values.reshape(485,1)
    tot_sdev_annual = np.array((np.sqrt(((w.T@Beta)@(Sigma_F@Rho_F@Sigma_F)@(Beta.T@w) + (w.T@(Sigma_epsilon)@w)*12)).round(3)))[0][0]
    sys_sdev_annual = np.array((np.sqrt((w.T@Beta)@(Sigma_F@Rho_F@Sigma_F)@(Beta.T@w))).round(3))[0][0]
    unsys_sdev_annual = np.array((np.sqrt((w.T@(Sigma_epsilon)@w)*12)).round(3))[0][0]
    pct_sys = ((sys_sdev_annual**2)/(tot_sdev_annual**2)).round(3)
    pct_unsys = ((sys_sdev_annual**2)/(tot_sdev_annual**2)).round(3)
  # DO NOT CHANGE CODE BELOW HERE
  # Note: for the function to work, you must calculate the values in the dictionary
  # using the exact variable names below
  # FRIENDLY REMINDER-- the values are in annualized terms
    summary_dictionary = {'tot_sdev_annual': tot_sdev_annual,
                          'sys_sdev_annual': sys_sdev_annual,
                          'unsys_sdev_annual': unsys_sdev_annual,
                          'pct_sys' : pct_sys,
                          'pct_unsys' : pct_unsys,
                          'beta_mkt' : beta_mkt,
                          'beta_smb' : beta_smb,
                          'beta_hml' : beta_hml
                         }

    df_summary = pd.DataFrame.from_dict(data=summary_dictionary, orient='index', columns=[summary_label])

    return df_summary




In [17]:
# *****************
# FINAL CELL
# *****************

# This will be the final cell. 
# NOTE:  You will need to create the appropriate values for the second argument.
# It may be a dataframe with JUST ONE column of weights or a numpy array (i.e. 1-d vector).
# So, for example: means YOUR dataframe or numpy array with portfolio weights

port_wt =  df_digest['PORTWT']# you need to replace this value  .. perhaps:  portwt=df_digest['PORTWT'], for example
bmk_wt = df_digest['BMKTWT']  # you need to replace this value
active_wt = df_digest['ACTWT'] # you need to replace this value 

# After you have prepared the arguments the following will be called

try:
  df_port_summary = calc_summary('Portfolio', port_wt, Beta, Sigma_F, Rho_F, Sigma_epsilon)
  df_bmk_summary = calc_summary('Benchmark', bmk_wt,   Beta, Sigma_F, Rho_F, Sigma_epsilon)
  df_active_summary = calc_summary('Active', active_wt, Beta, Sigma_F, Rho_F, Sigma_epsilon)
  df_all_combined = pd.concat([df_port_summary, df_bmk_summary, df_active_summary], axis=1)
  print(df_all_combined)
except:
  print('****************************************')
  print('*** ERROR *** : Last cell is not working')
  print('****************************************')


                   Portfolio  Benchmark  Active
tot_sdev_annual       15.843     16.157   4.600
sys_sdev_annual       15.657     15.964   3.229
unsys_sdev_annual      2.424      2.488   3.276
pct_sys                0.977      0.976   0.493
pct_unsys              0.977      0.976   0.493
beta_mkt               0.930      1.038  -0.109
beta_smb               0.086     -0.174   0.261
beta_hml               0.061     -0.131   0.192
